# 04 - Context API: Data Flow Between Nodes

## 🎯 The Problem: Passing DataFrames Without Globals

**Bad approach:**
```python
# Global state - hard to test, race conditions
global_data = {}

def node_a():
    global_data['customers'] = pd.read_csv('customers.csv')

def node_b():
    df = global_data['customers']  # Implicit dependency!
```

**Problems:**
- Hidden dependencies (what data does node_b need?)
- Hard to test in isolation
- Race conditions in parallel execution
- Memory leaks (when to clear?)

**Odibi's solution:** Explicit context passed to each node.

## 🦉 First Principles

### Dependency Injection
Don't access global state - receive dependencies as parameters.

```python
# Instead of:
def transform(): return global_data['input']

# Do:
def transform(ctx: Context): return ctx.get('input')
```

### Interface Segregation
Context provides minimal interface: register, get, has, list_names, clear

### Polymorphism
Same interface works for Pandas (dict storage) and Spark (temp views)

## 🔍 Reading Odibi Code: context.py

Let's analyze the actual implementation from Odibi.

In [ ]:
import sys
sys.path.insert(0, r'c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Odibi')

from odibi.context import Context, PandasContext, SparkContext, create_context
import pandas as pd
import inspect

# View the Context ABC
print("Context ABC Methods:")
print(inspect.getsource(Context))

### Key Design Points

1. **Abstract Base Class**: Forces all implementations to provide the 5 core methods
2. **Type Hints**: `Any` allows both Pandas and Spark DataFrames
3. **Explicit Errors**: `KeyError` with helpful messages shows available names
4. **Lifecycle**: `clear()` enables cleanup between runs

In [ ]:
# Examine PandasContext implementation
print("PandasContext Implementation:")
print(inspect.getsource(PandasContext))

### PandasContext Analysis

**Storage**: Simple dict `self._data: Dict[str, pd.DataFrame]`

**Type Safety**: 
```python
if not isinstance(df, pd.DataFrame):
    raise TypeError(f"Expected pandas.DataFrame, got {type(df)}")
```

**Error Messages**: Shows available names when key not found
```python
available = ", ".join(self._data.keys()) if self._data else "none"
raise KeyError(f"DataFrame '{name}' not found. Available: {available}")
```

In [ ]:
# Test PandasContext
ctx = PandasContext()

# Register some data
df = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
ctx.register('test_data', df)

print("Registered:", ctx.list_names())
print("Has 'test_data':", ctx.has('test_data'))
print("Retrieved:\n", ctx.get('test_data'))

# Try to get non-existent data
try:
    ctx.get('missing')
except KeyError as e:
    print(f"\nExpected error: {e}")

In [ ]:
# Examine SparkContext
print("SparkContext Implementation:")
print(inspect.getsource(SparkContext))

### SparkContext Analysis

**Storage**: Temp views in Spark catalog (not in-memory dict!)
```python
df.createOrReplaceTempView(name)
self._registered_views.add(name)  # Track names only
```

**Retrieval**: Query from catalog
```python
return self.spark.table(name)
```

**Cleanup**: Drop temp views
```python
self.spark.catalog.dropTempView(name)
```

**Why different from Pandas?**
- Spark DataFrames are lazy (execution plans)
- Temp views integrate with Spark SQL catalog
- Avoid serialization overhead

In [ ]:
# Factory pattern
print("create_context Factory:")
print(inspect.getsource(create_context))

# Use the factory
pandas_ctx = create_context('pandas')
print(f"\nCreated: {type(pandas_ctx).__name__}")

# Spark requires session
try:
    spark_ctx = create_context('spark')
except ValueError as e:
    print(f"Expected error: {e}")

## 🏗️ Build: CachingContext with LRU Eviction

Let's extend PandasContext with:
- Max size limit
- LRU (Least Recently Used) eviction
- Access tracking

In [ ]:
from collections import OrderedDict
from typing import Dict
import pandas as pd

class CachingContext(Context):
    """Context with LRU eviction policy."""
    
    def __init__(self, max_size: int = 10):
        """
        Args:
            max_size: Maximum number of DataFrames to cache
        """
        self.max_size = max_size
        self._data: OrderedDict[str, pd.DataFrame] = OrderedDict()
        self._eviction_count = 0
    
    def register(self, name: str, df: pd.DataFrame) -> None:
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"Expected pandas.DataFrame, got {type(df)}")
        
        # Remove if exists (to update position)
        if name in self._data:
            del self._data[name]
        
        # Evict LRU if at capacity
        if len(self._data) >= self.max_size:
            evicted_name, _ = self._data.popitem(last=False)  # FIFO = LRU
            self._eviction_count += 1
            print(f"⚠️  Evicted '{evicted_name}' (LRU)")
        
        # Add to end (most recent)
        self._data[name] = df
    
    def get(self, name: str) -> pd.DataFrame:
        if name not in self._data:
            available = ", ".join(self._data.keys()) if self._data else "none"
            raise KeyError(f"DataFrame '{name}' not found. Available: {available}")
        
        # Move to end (mark as recently used)
        self._data.move_to_end(name)
        return self._data[name]
    
    def has(self, name: str) -> bool:
        return name in self._data
    
    def list_names(self) -> list[str]:
        return list(self._data.keys())
    
    def clear(self) -> None:
        self._data.clear()
        self._eviction_count = 0
    
    def stats(self) -> Dict[str, int]:
        """Get cache statistics."""
        return {
            'size': len(self._data),
            'max_size': self.max_size,
            'evictions': self._eviction_count
        }

print("✅ CachingContext implemented")

In [ ]:
# Test LRU eviction
ctx = CachingContext(max_size=3)

# Register 3 DataFrames
for i in range(3):
    ctx.register(f'df{i}', pd.DataFrame({'x': [i]}))

print("After 3 registers:", ctx.list_names())
print("Stats:", ctx.stats())

# Access df0 (make it recent)
_ = ctx.get('df0')
print("\nAfter accessing df0:", ctx.list_names())

# Register df3 - should evict df1 (least recent)
print("\nRegistering df3...")
ctx.register('df3', pd.DataFrame({'x': [3]}))
print("After df3:", ctx.list_names())
print("Stats:", ctx.stats())

# Verify df1 was evicted
print(f"\nHas df1: {ctx.has('df1')}")

## ✅ Testing: Context Isolation

Critical pattern: Each test gets a fresh context.

In [ ]:
import pytest

@pytest.fixture
def fresh_context():
    """Provide isolated context for each test."""
    ctx = PandasContext()
    yield ctx
    ctx.clear()  # Cleanup after test

# Simulated tests (run with pytest)
def test_register_and_get(fresh_context):
    df = pd.DataFrame({'a': [1, 2]})
    fresh_context.register('test', df)
    
    retrieved = fresh_context.get('test')
    assert retrieved.equals(df)

def test_isolation(fresh_context):
    # This test gets a NEW context (empty)
    assert fresh_context.list_names() == []

print("✅ Test patterns defined")
print("Run with: pytest lesson.ipynb")

## 💾 Memory Management Patterns

### Pattern 1: Context as Context Manager

In [ ]:
from contextlib import contextmanager

@contextmanager
def managed_context(engine='pandas'):
    """Auto-cleanup context."""
    ctx = create_context(engine)
    try:
        yield ctx
    finally:
        ctx.clear()
        print("🧹 Context cleaned")

# Usage
with managed_context() as ctx:
    ctx.register('temp', pd.DataFrame({'x': [1, 2, 3]}))
    print("Inside:", ctx.list_names())

# Outside context - data is cleared
print("Outside: (ctx cleared)")

### Pattern 2: Checkpoint and Restore

In [ ]:
class CheckpointableContext(PandasContext):
    """Context that can save/restore state."""
    
    def __init__(self):
        super().__init__()
        self._checkpoints = []
    
    def checkpoint(self) -> None:
        """Save current state."""
        # Deep copy the data dict
        self._checkpoints.append({k: v.copy() for k, v in self._data.items()})
    
    def restore(self) -> None:
        """Restore last checkpoint."""
        if not self._checkpoints:
            raise RuntimeError("No checkpoints available")
        self._data = self._checkpoints.pop()

# Demo
ctx = CheckpointableContext()
ctx.register('initial', pd.DataFrame({'x': [1]}))
print("Initial:", ctx.list_names())

ctx.checkpoint()
ctx.register('added', pd.DataFrame({'y': [2]}))
print("After adding:", ctx.list_names())

ctx.restore()
print("After restore:", ctx.list_names())

## 🎯 Context in Transform Functions

How context flows through a pipeline:

In [ ]:
# Typical Odibi node pattern
def load_customers(ctx: Context) -> pd.DataFrame:
    """Load and register customers."""
    df = pd.DataFrame({
        'customer_id': [1, 2, 3],
        'name': ['Alice', 'Bob', 'Carol']
    })
    ctx.register('customers', df)
    return df

def load_orders(ctx: Context) -> pd.DataFrame:
    """Load and register orders."""
    df = pd.DataFrame({
        'order_id': [101, 102],
        'customer_id': [1, 2],
        'amount': [100, 200]
    })
    ctx.register('orders', df)
    return df

def join_data(ctx: Context) -> pd.DataFrame:
    """Join customers and orders."""
    customers = ctx.get('customers')  # Explicit dependency!
    orders = ctx.get('orders')
    
    result = orders.merge(customers, on='customer_id')
    ctx.register('customer_orders', result)
    return result

# Execute pipeline
ctx = PandasContext()
load_customers(ctx)
load_orders(ctx)
result = join_data(ctx)

print("Pipeline result:")
print(result)
print("\nContext contains:", ctx.list_names())

## 🎓 Summary

### Context Design Principles
1. **Explicit dependencies**: No globals, pass context to functions
2. **Polymorphic**: Same interface for Pandas/Spark
3. **Isolated**: Fresh context per pipeline run
4. **Lifecycle management**: Clear when done

### Implementation Strategies
- **PandasContext**: Dict-based, simple and fast
- **SparkContext**: Temp views, integrates with catalog
- **Extensions**: Caching, checkpoints, LRU eviction

### Best Practices
- Use `has()` before `get()` for optional data
- Always `clear()` in test teardown
- Prefer context managers for auto-cleanup
- Name DataFrames descriptively

### Next Steps
→ `exercises.ipynb` - Practice implementing context features  
→ `context_patterns.md` - Best practices reference